<a href="https://colab.research.google.com/github/Un0ptimized/smaug/blob/develop/ChequeAcc_TranHistory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Extract zip file en save files
import zipfile
import pandas as pd
import os 

# Extract zip file
with zipfile.ZipFile('/content/drive/MyDrive/Data Lake/Finances/Transaction History/transaction_history_Cheque_Account.zip','r') as zip:
  zip.extractall('/content/drive/MyDrive/Data Lake/Finances/Transaction History/FNB Cheque Account/Raw Data/')

# Verander extracted filename na vandag se datum
unzipped_file = '/content/drive/MyDrive/Data Lake/Finances/Transaction History/FNB Cheque Account/Raw Data/62385729771.csv'
rawfile = os.rename('/content/drive/MyDrive/Data Lake/Finances/Transaction History/FNB Cheque Account/Raw Data/62385729771.csv', 
                    f'/content/drive/MyDrive/Data Lake/Finances/Transaction History/FNB Cheque Account/Raw Data/62385729771_{pd.Timestamp("now").strftime("%Y-%m-%d")}.csv')
# Drop die zip file.
!rm '/content/drive/MyDrive/Data Lake/Finances/Transaction History/transaction_history_Cheque_Account.zip'


In [ ]:
# Combine al die csv files
import numpy as np
import glob
from datetime import datetime as dt

path = r'/content/drive/MyDrive/Data Lake/Finances/Transaction History/FNB Cheque Account/Raw Data/'
all_files = glob.glob(path + '/*.csv')

# Sit elke file in 'n dataframe en concat dan df's aanmekaar
df_each_file = (pd.read_csv(f, sep=', ', header=4 ,parse_dates=['Date'] ,engine='python') for f in all_files)
df = pd.concat(df_each_file, ignore_index=True)

# Concat date en description om unique key te kry
df['Key'] = df['Date'].dt.strftime('%Y-%m-%d') + df['Amount'].astype(str) + df['Description']
# Remove die duplicates op die key kolom
df = df.drop_duplicates(subset= ['Key'])


In [ ]:
# Add kolomme om data diepte te verbeter
# Create kolom vir Inkomste/Uitgawes om te kan filter 
df['Inkomste/Uitgawe'] = np.where(df['Amount'] < 0, 'Uitgawe', 'Inkomste')

# Create kolom vir Transaction Type
df.loc[(df['Description'].str.startswith(pat= 'PURCH')), 'Transaction Type'] = 'Cash'
df.loc[(df['Description'].str.startswith(pat= 'ATM CASH')), 'Transaction Type'] = 'ATM'
df.loc[(df['Description'].str.startswith(pat= 'FNB APP TRANSFER')), 'Transaction Type'] = 'Transfer'
df.loc[(df['Description'].str.startswith(pat= 'BYC DEBIT')), 'Transaction Type'] = 'Transfer'
df.loc[(df['Description'].str.startswith(pat= '#INT PYMT FEE')), 'Transaction Type'] = 'Bank Fees'
df.loc[(df['Description'].str.startswith(pat= 'SEND')), 'Transaction Type'] = 'E-Wallet'
df.loc[(df['Description'].str.contains(pat= 'APPLE.COM')), 'Transaction Type'] = 'Debit Order'
df.loc[(df['Description'].str.startswith(pat= 'Takealo*t')), 'Transaction Type'] = 'Online'
df.loc[(df['Description'].str.startswith(pat= 'ELECTRICITY')), 'Transaction Type'] = 'Online'
df.loc[(df['Description'].str.startswith(pat= 'DISC PREM')), 'Transaction Type'] = 'Debit Order'
df.loc[(df['Description'].str.startswith(pat= 'SBSA VAF')), 'Transaction Type'] = 'Debit Order'
df.loc[(df['Description'].str.startswith(pat= 'SANTAM')), 'Transaction Type'] = 'Debit Order'
df.loc[(df['Description'].str.startswith(pat= 'CARTRACK')), 'Transaction Type'] = 'Debit Order'
df.loc[(df['Description'].str.startswith(pat= '#SERVICE FEES')), 'Transaction Type'] = 'Bank Fees'
df.loc[(df['Description'].str.startswith(pat= '#MONTHLY ACCOUNT FEE')), 'Transaction Type'] = 'Bank Fees'
df.loc[(df['Description'].str.startswith(pat= 'FNBINVEST')), 'Transaction Type'] = 'Debit Order'
df.loc[(df['Description'].str.startswith(pat= 'TEMP')), 'Transaction Type'] = 'Transfer'
df.loc[(df['Description'].str.startswith(pat= 'PROPERTY TAX')), 'Transaction Type'] = 'Debit Order'
df.loc[(df['Description'].str.startswith(pat= 'FUSION INTEREST')), 'Transaction Type'] = 'Bank Fees'
df.loc[(df['Description'].str.startswith(pat= 'MED-TERM SAVINGS')), 'Transaction Type'] = 'Transfer'
df.loc[(df['Description'].str.startswith(pat= '#MONTHLY CREDIT FEE')), 'Transaction Type'] = 'Bank Fees'
df.loc[(df['Description'].str.startswith(pat= 'DR INTEREST')), 'Transaction Type'] = 'Bank Fees'
df.loc[(df['Description'].str.startswith(pat= 'INT ON DEBIT BALANCE')), 'Transaction Type'] = 'Bank Fees'
df.loc[(df['Description'].str.startswith(pat= 'M B printing')), 'Transaction Type'] = 'Cash'
df.loc[(df['Description'].str.startswith(pat= 'FNB CARD')), 'Transaction Type'] = 'Transfer'
df.loc[(df['Description'].str.startswith(pat= 'FNB HOME')), 'Transaction Type'] = 'Debit Order'
df.loc[(df['Description'].str.startswith(pat= '99.99 SPOTIFY')), 'Transaction Type'] = 'Debit Order'
df.loc[(df['Description'].str.startswith(pat= 'Gautrain CTLS')), 'Transaction Type'] = 'Online'
df.loc[(df['Description'].str.startswith(pat= 'MTN SP')), 'Transaction Type'] = 'Debit Order'
df.loc[(df['Description'].str.startswith(pat= 'KREDIETKAART')), 'Transaction Type'] = 'Transfer'
df.loc[(df['Description'].str.startswith(pat= '#DEBIT ORDER DISPUTE')), 'Transaction Type'] = 'Bank Fees'
df.loc[(df['Description'].str.startswith(pat= 'DECLINED FOREIGN')), 'Transaction Type'] = 'Bank Fees'
df.loc[(df['Description'].str.startswith(pat= 'MAGTAPE UNPAID')), 'Transaction Type'] = 'Reversal'
# Fill NaN values met EFT
df['Transaction Type'].fillna('EFT', inplace= True)

# Create 'n kolom vir Sub-Kategorie 2
df.loc[(df['Description'].str.contains(pat= 'BYC DEBIT')), 'Sub-kategorie 2'] = 'BYC'
df.loc[(df['Description'].str.contains(pat= 'Tiny Town')), 'Sub-kategorie 2'] = 'Tiny Town'
df.loc[(df['Description'].str.contains(pat= 'PnP')), 'Sub-kategorie 2'] = 'Pick-n-Pay'
df.loc[(df['Description'].str.contains(pat= 'GERDA CAMPHER')), 'Sub-kategorie 2'] = 'Gerda - Ma'
df.loc[(df['Description'].str.contains(pat= 'FNB APP TRANSFER FROM FAM TAX')), 'Sub-kategorie 2'] = 'Gerda - Ma'
df.loc[(df['Description'].str.contains(pat= 'Italio Pizza')), 'Sub-kategorie 2'] = 'Italio Pizza'
df.loc[(df['Description'].str.contains(pat= 'Tops')), 'Sub-kategorie 2'] = 'Tops'
df.loc[(df['Description'].str.contains(pat= 'Clicks')), 'Sub-kategorie 2'] = 'Clicks'
df.loc[(df['Description'].str.contains(pat= 'ATM CASH')), 'Sub-kategorie 2'] = 'ATM'
df.loc[(df['Description'].str.contains(pat= 'NOOI')), 'Sub-kategorie 2'] = 'Anna'
df.loc[(df['Description'].str.contains(pat= 'ELECTRICITY')), 'Sub-kategorie 2'] = 'Elektrisiteit'
df.loc[(df['Description'].str.contains(pat= 'W&L')), 'Sub-kategorie 2'] = 'Water & Sanitation'
df.loc[(df['Description'].str.contains(pat= 'FNB APP TRANSFER FROM IMPACT ETC')), 'Sub-kategorie 2'] = 'Impact Deposito'
df.loc[(df['Description'].str.contains(pat= '#INT PYMT FEE')), 'Sub-kategorie 2'] = 'Service Fee'
df.loc[(df['Description'].str.contains(pat= 'SEND 27721165439')), 'Sub-kategorie 2'] = 'E-Wallet - Ma'
df.loc[(df['Description'].str.contains(pat= '14.99 APPLE.COM')), 'Sub-kategorie 2'] = 'Apple - Cloud'
df.loc[(df['Description'].str.contains(pat= 'MR D FOOD')), 'Sub-kategorie 2'] = 'Mr. D'
df.loc[(df['Description'].str.contains(pat= 'CAPITAL CRAFT')), 'Sub-kategorie 2'] = 'Capital Craft'
df.loc[(df['Description'].str.contains(pat= 'Takealo')), 'Sub-kategorie 2'] = 'Takealot'
df.loc[(df['Description'].str.contains(pat= 'Checkers Sixty60')), 'Sub-kategorie 2'] = 'Checkers Sixty60'
df.loc[(df['Description'].str.contains(pat= 'Clearance W')), 'Sub-kategorie 2'] = 'Clearance Warehouse'
df.loc[(df['Description'].str.contains(pat= 'WOOLWORTHS')), 'Sub-kategorie 2'] = 'Woolworths'
df.loc[(df['Description'].str.contains(pat= 'Ciao Bella')), 'Sub-kategorie 2'] = 'Cioa Bella'
df.loc[(df['Description'].str.contains(pat= 'Dischem')), 'Sub-kategorie 2'] = 'Dischem'
df.loc[(df['Description'].str.contains(pat= 'DISC PREM')), 'Sub-kategorie 2'] = 'Discovery - Medies'
df.loc[(df['Description'].str.contains(pat= 'SEND 27677201092')), 'Sub-kategorie 2'] = 'Yskas Repair'
df.loc[(df['Description'].str.contains(pat= 'FNB APP TRANSFER FROM HUIS')), 'Sub-kategorie 2'] = 'Huis Koop Expenses'
df.loc[(df['Description'].str.contains(pat= 'GAUTRAIN', case= False)), 'Sub-kategorie 2'] = 'Gautrain'
df.loc[(df['Description'].str.contains(pat= 'CHAMBERLAIN')), 'Sub-kategorie 2'] = 'Chamberlains'
df.loc[(df['Description'].str.contains(pat= 'COUCH')), 'Sub-kategorie 2'] = 'Couch - Transfer'
df.loc[(df['Description'].str.contains(pat= 'Telkom')), 'Sub-kategorie 2'] = 'Telkom'
df.loc[(df['Description'].str.contains(pat= 'FNB APP TRANSFER FROM KREDIET')), 'Sub-kategorie 2'] = 'Kredietkaart Transfers'
df.loc[(df['Description'].str.contains(pat= 'FNB APP TRANSFER FROM SAVE REVERSE')), 'Sub-kategorie 2'] = 'Kredietkaart Transfers'
df.loc[(df['Description'].str.contains(pat= 'FNB APP TRANSFER FROM INC CONN')), 'Sub-kategorie 2'] = 'Kredietkaart Transfers'
df.loc[(df['Description'].str.contains(pat= 'SBSA VAF')), 'Sub-kategorie 2'] = 'Standardbank Lening'
df.loc[(df['Description'].str.contains(pat= 'Makro')), 'Sub-kategorie 2'] = 'Makro'
df.loc[(df['Description'].str.contains(pat= 'FAT CAKE CITY')), 'Sub-kategorie 2'] = 'Fat Cake City'
df.loc[(df['Description'].str.contains(pat= 'SANTAM')), 'Sub-kategorie 2'] = 'Santam'
df.loc[(df['Description'].str.contains(pat= 'CARTRACK')), 'Sub-kategorie 2'] = 'Cartrack'
df.loc[(df['Description'].str.contains(pat= 'ENGEN')), 'Sub-kategorie 2'] = 'Engen'
df.loc[(df['Description'].str.contains(pat= 'PIZZA TOPPINGS')), 'Sub-kategorie 2'] = 'Romans'
df.loc[(df['Description'].str.contains(pat= 'HUUR')), 'Sub-kategorie 2'] = 'Huur'
df.loc[(df['Description'].str.contains(pat= '#SERVICE FEES')), 'Sub-kategorie 2'] = 'Service Fee'
df.loc[(df['Description'].str.contains(pat= '#MONTHLY ACCOUNT FEE')), 'Sub-kategorie 2'] = 'FNB Account Fee'
df.loc[(df['Description'].str.contains(pat= 'FNBINVEST')), 'Sub-kategorie 2'] = 'FNB Retirement Annuity'
df.loc[(df['Description'].str.contains(pat= 'TEMP')), 'Sub-kategorie 2'] = 'Kredietkaart Transfers'
df.loc[(df['Description'].str.contains(pat= 'PROPERTY TAX')), 'Sub-kategorie 2'] = 'Erfbelasting'
df.loc[(df['Description'].str.contains(pat= 'INTEREST REBATE')), 'Sub-kategorie 2'] = 'Rente'
df.loc[(df['Description'].str.contains(pat= 'MED-TERM SAVINGS')), 'Sub-kategorie 2'] = 'Med-term Savings'
df.loc[(df['Description'].str.contains(pat= '#MONTHLY CREDIT FEE')), 'Sub-kategorie 2'] = 'FNB Credit Fee'
df.loc[(df['Description'].str.contains(pat= 'INT ON DEBIT BALANCE')), 'Sub-kategorie 2'] = 'Rente'
df.loc[(df['Description'].str.contains(pat= 'M B printing')), 'Sub-kategorie 2'] = 'MB Printing'
df.loc[(df['Description'].str.contains(pat= 'FNB CARD')), 'Sub-kategorie 2'] = 'Kredietkaart Auto'
df.loc[(df['Description'].str.contains(pat= 'FNB HOME')), 'Sub-kategorie 2'] = 'FNB Lening'
df.loc[(df['Description'].str.contains(pat= 'IKH')), 'Sub-kategorie 2'] = 'IKH'
df.loc[(df['Description'].str.contains(pat= 'SALARY')), 'Sub-kategorie 2'] = 'Ilion'
df.loc[(df['Description'].str.contains(pat= 'PAYMENT FROM  HEIGERS')), 'Sub-kategorie 2'] = 'Gerda'
df.loc[(df['Description'].str.contains(pat= 'Checkers')), 'Sub-kategorie 2'] = 'Checkers'
df.loc[(df['Description'].str.contains(pat= 'LYNNWOOD HOUSEWIFE')), 'Sub-kategorie 2'] = 'Lynnwood Housewife'
df.loc[(df['Description'].str.contains(pat= 'FLM')), 'Sub-kategorie 2'] = 'Food Lovers'
df.loc[(df['Description'].str.contains(pat= 'VEGA MOTORS')), 'Sub-kategorie 2'] = 'Vega Motors'
df.loc[(df['Description'].str.contains(pat= 'Spar')), 'Sub-kategorie 2'] = 'Spar'
df.loc[(df['Description'].str.contains(pat= 'SPICE THE INDIAN KI')), 'Sub-kategorie 2'] = 'Indian Kitchen'
df.loc[(df['Description'].str.contains(pat= '29.99 APPLE.COM')), 'Sub-kategorie 2'] = 'Apple - GoogleOne'
df.loc[(df['Description'].str.contains(pat= '99.99 SPOTIFY')), 'Sub-kategorie 2'] = 'Spotify'
df.loc[(df['Description'].str.contains(pat= 'Fudio PTY')), 'Sub-kategorie 2'] = 'Fudio'
df.loc[(df['Description'].str.contains(pat= 'PURCH BK')), 'Sub-kategorie 2'] = 'Burger King'
df.loc[(df['Description'].str.contains(pat= 'Coffee Cult')), 'Sub-kategorie 2'] = 'Coffee Cult'
df.loc[(df['Description'].str.contains(pat= 'Dwelling de')), 'Sub-kategorie 2'] = '"Dwelling" Not Sure Where'
df.loc[(df['Description'].str.contains(pat= 'Internation')), 'Sub-kategorie 2'] = '"Internation" Not Sure Where'
df.loc[(df['Description'].str.contains(pat= 'PBF')), 'Sub-kategorie 2'] = 'Huurinkomste'
df.loc[(df['Description'].str.contains(pat= 'TIME SQUARE CASINO')), 'Sub-kategorie 2'] = 'Time Square Parking'
df.loc[(df['Description'].str.contains(pat= 'BERTIER')), 'Sub-kategorie 2'] = 'Bertie'
df.loc[(df['Description'].str.contains(pat= 'VOVO TELO')), 'Sub-kategorie 2'] = 'Vovo Telo'
df.loc[(df['Description'].str.contains(pat= 'Cafe Kahvi')), 'Sub-kategorie 2'] = 'Cafe Kahvi'
df.loc[(df['Description'].str.contains(pat= 'PAYMENT FROM  PA')), 'Sub-kategorie 2'] = 'Ontvangs - Pa'
df.loc[(df['Description'].str.contains(pat= 'Incredible Connecti')), 'Sub-kategorie 2'] = 'Incredible Connection'
df.loc[(df['Description'].str.contains(pat= 'CONTINENTAL LINEN')), 'Sub-kategorie 2'] = 'Continental Linen'
df.loc[(df['Description'].str.contains(pat= 'WEST PACK PETZONE')), 'Sub-kategorie 2'] = 'Petzone'
df.loc[(df['Description'].str.contains(pat= 'TSAFRIKA HEADOFFICE')), 'Sub-kategorie 2'] = 'EY Lunch'
df.loc[(df['Description'].str.contains(pat= 'MTN SP')), 'Sub-kategorie 2'] = 'MTN'
df.loc[(df['Description'].str.contains(pat= 'PlaystationNetwork')), 'Sub-kategorie 2'] = 'Playstation Network'
df.loc[(df['Description'].str.contains(pat= 'APP PAYMENT FROM  SKULD')), 'Sub-kategorie 2'] = 'Huur'
df.loc[(df['Description'].str.contains(pat= 'GUARDRISK')), 'Sub-kategorie 2'] = 'Guardrisk'
df.loc[(df['Description'].str.contains(pat= 'IC COFFEE')), 'Sub-kategorie 2'] = 'IC Coffee'
df.loc[(df['Description'].str.contains(pat= 'HUIS TRANSFER COSTS')), 'Sub-kategorie 2'] = 'Huis Transfer Costs'
df.loc[(df['Description'].str.contains(pat= 'KREDIETKAART')), 'Sub-kategorie 2'] = 'Kredietkaart Transfers'
df.loc[(df['Description'].str.contains(pat= 'ABSA BANK Ilion')), 'Sub-kategorie 2'] = 'Ilion'
df.loc[(df['Description'].str.contains(pat= 'HENNIE S')), 'Sub-kategorie 2'] = 'Hennies'
df.loc[(df['Description'].str.contains(pat= 'Bolt')), 'Sub-kategorie 2'] = 'Bolt'
df.loc[(df['Description'].str.contains(pat= 'BP EXPRESS')), 'Sub-kategorie 2'] = 'BP'
df.loc[(df['Description'].str.contains(pat= '#DEBIT ORDER DISPUTE FEE')), 'Sub-kategorie 2'] = 'Debit Order Dispute Fee'
df.loc[(df['Description'].str.contains(pat= 'PADSTAL')), 'Sub-kategorie 2'] = 'Padstal'
df.loc[(df['Description'].str.contains(pat= 'DECLINED FOREIGN')), 'Sub-kategorie 2'] = 'Declined Foreign Transaction'
df.loc[(df['Description'].str.contains(pat= 'MAGTAPE UNPAID')), 'Sub-kategorie 2'] = 'Reversal'
df.loc[(df['Description'].str.contains(pat= 'MIDAS')), 'Sub-kategorie 2'] = 'Midas'
df.loc[(df['Description'].str.contains(pat= 'Debonairs Pizza')), 'Sub-kategorie 2'] = 'Debonairs'
# Fill NaN values met Weet Nie
df['Sub-kategorie 2'].fillna('Weet Nie', inplace= True)

# Create 'n kolom vir Sub-Kategorie
df.loc[(df['Sub-kategorie 2'].str.contains(pat= 'BYC')), 'Sub-kategorie'] = 'BYC'
df.loc[(df['Sub-kategorie 2'].str.contains(pat= 'Debonairs')), 'Sub-kategorie'] = 'Kitskos'
df.loc[(df['Sub-kategorie 2'].str.contains(pat= 'Tiny Town')), 'Sub-kategorie'] = 'Kruie'
df.loc[(df['Sub-kategorie 2'].str.contains(pat= 'Pick-n-Pay')), 'Sub-kategorie'] = 'Pick-n-Pay'
df.loc[(df['Sub-kategorie 2'].str.contains(pat= 'Gerda - Ma')), 'Sub-kategorie'] = 'Ma'
df.loc[(df['Sub-kategorie 2'].str.contains(pat= 'Italio Pizza')), 'Sub-kategorie'] = 'Kitskos'
df.loc[(df['Sub-kategorie 2'].str.contains(pat= 'Tops')), 'Sub-kategorie'] = 'Alkoholiese Verversings'
df.loc[(df['Sub-kategorie 2'].str.contains(pat= 'Clicks')), 'Sub-kategorie'] = 'Clicks'
df.loc[(df['Sub-kategorie 2'].str.contains(pat= 'ATM')), 'Sub-kategorie'] = 'ATM'
df.loc[(df['Sub-kategorie 2'].str.contains(pat= 'Anna')), 'Sub-kategorie'] = 'Anna'
df.loc[(df['Sub-kategorie 2'].str.contains(pat= 'Elektrisiteit')), 'Sub-kategorie'] = 'Elektrisiteit'
df.loc[(df['Sub-kategorie 2'].str.contains(pat= 'Water & Sanitation')), 'Sub-kategorie'] = 'Water & Sanitation'
df.loc[(df['Sub-kategorie 2'].str.contains(pat= 'Impact Deposito')), 'Sub-kategorie'] = 'Depositos'
df.loc[(df['Sub-kategorie 2'].str.contains(pat= 'Service Fee')), 'Sub-kategorie'] = 'Service Fee'
df.loc[(df['Sub-kategorie 2'].str.contains(pat= 'E-Wallet - Ma')), 'Sub-kategorie'] = 'Ma'
df.loc[(df['Sub-kategorie 2'].str.contains(pat= 'Apple - Cloud')), 'Sub-kategorie'] = 'Apple'
df.loc[(df['Sub-kategorie 2'].str.contains(pat= 'Mr. D')), 'Sub-kategorie'] = 'Kitskos'
df.loc[(df['Sub-kategorie 2'].str.contains(pat= 'Capital Craft')), 'Sub-kategorie'] = 'Kitskos'
df.loc[(df['Sub-kategorie 2'].str.contains(pat= 'Takealot')), 'Sub-kategorie'] = 'Takealot'
df.loc[(df['Sub-kategorie 2'].str.contains(pat= 'Checkers')), 'Sub-kategorie'] = 'Checkers'
df.loc[(df['Sub-kategorie 2'].str.contains(pat= 'Clearance Warehouse')), 'Sub-kategorie'] = 'Couch'
df.loc[(df['Sub-kategorie 2'].str.contains(pat= 'Woolworths')), 'Sub-kategorie'] = 'Woolworths'
df.loc[(df['Sub-kategorie 2'].str.contains(pat= 'Cioa Bella')), 'Sub-kategorie'] = 'Kitskos'
df.loc[(df['Sub-kategorie 2'].str.contains(pat= 'Dischem')), 'Sub-kategorie'] = 'Dischem'
df.loc[(df['Sub-kategorie 2'].str.contains(pat= 'Discovery - Medies')), 'Sub-kategorie'] = 'Discovery'
df.loc[(df['Sub-kategorie 2'].str.contains(pat= 'Yskas Repair')), 'Sub-kategorie'] = 'Yskas Repair'
df.loc[(df['Sub-kategorie 2'].str.contains(pat= 'Huis Koop Expenses')), 'Sub-kategorie'] = 'Huis Koop Expenses'
df.loc[(df['Sub-kategorie 2'].str.contains(pat= 'Gautrain')), 'Sub-kategorie'] = 'Gautrain'
df.loc[(df['Sub-kategorie 2'].str.contains(pat= 'Chamberlains')), 'Sub-kategorie'] = 'DIY'
df.loc[(df['Sub-kategorie 2'].str.contains(pat= 'Couch - Transfer')), 'Sub-kategorie'] = 'Couch'
df.loc[(df['Sub-kategorie 2'].str.contains(pat= 'Telkom')), 'Sub-kategorie'] = 'Internet'
df.loc[(df['Sub-kategorie 2'].str.contains(pat= 'Kredietkaart Transfers')), 'Sub-kategorie'] = 'Kredietkaart'
df.loc[(df['Sub-kategorie 2'].str.contains(pat= 'Standardbank Lening')), 'Sub-kategorie'] = 'Kar Lening'
df.loc[(df['Sub-kategorie 2'].str.contains(pat= 'Makro')), 'Sub-kategorie'] = 'Makro'
df.loc[(df['Sub-kategorie 2'].str.contains(pat= 'Fat Cake City')), 'Sub-kategorie'] = 'Kitskos'
df.loc[(df['Sub-kategorie 2'].str.contains(pat= 'Santam')), 'Sub-kategorie'] = 'Santam'
df.loc[(df['Sub-kategorie 2'].str.contains(pat= 'Cartrack')), 'Sub-kategorie'] = 'Cartrack'
df.loc[(df['Sub-kategorie 2'].str.contains(pat= 'Engen')), 'Sub-kategorie'] = 'Petrol'
df.loc[(df['Sub-kategorie 2'].str.contains(pat= 'Romans')), 'Sub-kategorie'] = 'Kitskos'
df.loc[(df['Sub-kategorie 2'].str.contains(pat= 'Huur')), 'Sub-kategorie'] = 'Huur'
df.loc[(df['Sub-kategorie 2'].str.contains(pat= 'FNB Account Fee')), 'Sub-kategorie'] = 'FNB Account Fee'
df.loc[(df['Sub-kategorie 2'].str.contains(pat= 'FNB Retirement Annuity')), 'Sub-kategorie'] = 'FNB RA'
df.loc[(df['Sub-kategorie 2'].str.contains(pat= 'Erfbelasting')), 'Sub-kategorie'] = 'Erfbelasting'
df.loc[(df['Sub-kategorie 2'].str.contains(pat= 'Rente')), 'Sub-kategorie'] = 'Rente'
df.loc[(df['Sub-kategorie 2'].str.contains(pat= 'MB Printing')), 'Sub-kategorie'] = 'MB Printing'
df.loc[(df['Sub-kategorie 2'].str.contains(pat= 'Kredietkaart Auto')), 'Sub-kategorie'] = 'Kredietkaart'
df.loc[(df['Sub-kategorie 2'].str.contains(pat= 'FNB Lening')), 'Sub-kategorie'] = 'Huis Lening'
df.loc[(df['Sub-kategorie 2'].str.contains(pat= 'IKH')), 'Sub-kategorie'] = 'IKH'
df.loc[(df['Sub-kategorie 2'].str.contains(pat= 'Ilion')), 'Sub-kategorie'] = 'Ilion'
df.loc[(df['Sub-kategorie 2'].str.contains(pat= 'Gerda')), 'Sub-kategorie'] = 'Ma'
df.loc[(df['Sub-kategorie 2'].str.contains(pat= 'Lynnwood Housewife')), 'Sub-kategorie'] = 'Lynnwood Housewife'
df.loc[(df['Sub-kategorie 2'].str.contains(pat= 'Food Lovers')), 'Sub-kategorie'] = 'Food Lovers'
df.loc[(df['Sub-kategorie 2'].str.contains(pat= 'Vega Motors')), 'Sub-kategorie'] = 'Vega Motors'
df.loc[(df['Sub-kategorie 2'].str.contains(pat= 'Spar')), 'Sub-kategorie'] = 'Spar'
df.loc[(df['Sub-kategorie 2'].str.contains(pat= 'Med-term Savings')), 'Sub-kategorie'] = 'Med-term Savings'
df.loc[(df['Sub-kategorie 2'].str.contains(pat= 'FNB Credit Fee')), 'Sub-kategorie'] = 'FNB Credit Fee'
df.loc[(df['Sub-kategorie 2'].str.contains(pat= 'Indian Kitchen')), 'Sub-kategorie'] = 'Kitskos'
df.loc[(df['Sub-kategorie 2'].str.contains(pat= 'Apple - GoogleOne')), 'Sub-kategorie'] = 'Apple'
df.loc[(df['Sub-kategorie 2'].str.contains(pat= 'Spotify')), 'Sub-kategorie'] = 'Spotify'
df.loc[(df['Sub-kategorie 2'].str.contains(pat= 'Fudio')), 'Sub-kategorie'] = 'Kitskos'
df.loc[(df['Sub-kategorie 2'].str.contains(pat= 'Burger King')), 'Sub-kategorie'] = 'Kitskos'
df.loc[(df['Sub-kategorie 2'].str.contains(pat= 'Coffee Cult')), 'Sub-kategorie'] = 'Kitskos'
df.loc[(df['Sub-kategorie 2'].str.contains(pat= '"Dwelling" Not Sure Where')), 'Sub-kategorie'] = 'Unknown Misc'
df.loc[(df['Sub-kategorie 2'].str.contains(pat= '"Internation" Not Sure Where')), 'Sub-kategorie'] = 'Unknown Misc'
df.loc[(df['Sub-kategorie 2'].str.contains(pat= 'Huurinkomste')), 'Sub-kategorie'] = 'Huurinkomste'
df.loc[(df['Sub-kategorie 2'].str.contains(pat= 'Time Square Parking')), 'Sub-kategorie'] = 'Time Square Parking'
df.loc[(df['Sub-kategorie 2'].str.contains(pat= 'Bertie')), 'Sub-kategorie'] = 'Bertie'
df.loc[(df['Sub-kategorie 2'].str.contains(pat= 'Vovo Telo')), 'Sub-kategorie'] = 'Kitskos'
df.loc[(df['Sub-kategorie 2'].str.contains(pat= 'Cafe Kahvi')), 'Sub-kategorie'] = 'Kitskos'
df.loc[(df['Sub-kategorie 2'].str.contains(pat= 'Ontvangs - Pa')), 'Sub-kategorie'] = 'Pa'
df.loc[(df['Sub-kategorie 2'].str.contains(pat= 'Incredible Connection')), 'Sub-kategorie'] = 'Incredible Connection'
df.loc[(df['Sub-kategorie 2'].str.contains(pat= 'Continental Linen')), 'Sub-kategorie'] = 'Continental Linen'
df.loc[(df['Sub-kategorie 2'].str.contains(pat= 'Petzone')), 'Sub-kategorie'] = 'Petzone'
df.loc[(df['Sub-kategorie 2'].str.contains(pat= 'EY Lunch')), 'Sub-kategorie'] = 'Kitskos'
df.loc[(df['Sub-kategorie 2'].str.contains(pat= 'MTN')), 'Sub-kategorie'] = 'MTN'
df.loc[(df['Sub-kategorie 2'].str.contains(pat= 'Playstation Network')), 'Sub-kategorie'] = 'Playstation Network'
df.loc[(df['Sub-kategorie 2'].str.contains(pat= 'Guardrisk')), 'Sub-kategorie'] = 'Guardrisk'
df.loc[(df['Sub-kategorie 2'].str.contains(pat= 'IC Coffee')), 'Sub-kategorie'] = 'Kitskos'
df.loc[(df['Sub-kategorie 2'].str.contains(pat= 'Huis Transfer Costs')), 'Sub-kategorie'] = 'Huis Koop Expenses'
df.loc[(df['Sub-kategorie 2'].str.contains(pat= 'Hennies')), 'Sub-kategorie'] = 'Kitskos'
df.loc[(df['Sub-kategorie 2'].str.contains(pat= 'Bolt')), 'Sub-kategorie'] = 'Bolt'
df.loc[(df['Sub-kategorie 2'].str.contains(pat= 'BP')), 'Sub-kategorie'] = 'Petrol'
df.loc[(df['Sub-kategorie 2'].str.contains(pat= 'Debit Order Dispute Fee')), 'Sub-kategorie'] = 'Debit Order Dispute Fee'
df.loc[(df['Sub-kategorie 2'].str.contains(pat= 'Padstal')), 'Sub-kategorie'] = 'Kitskos'
df.loc[(df['Sub-kategorie 2'].str.contains(pat= 'Declined Foreign Transaction')), 'Sub-kategorie'] = 'Declined Foreign Transaction'
df.loc[(df['Sub-kategorie 2'].str.contains(pat= 'Reversal')), 'Sub-kategorie'] = 'Reversal'
df.loc[(df['Sub-kategorie 2'].str.contains(pat= 'Midas')), 'Sub-kategorie'] = 'Midas'
df.loc[(df['Sub-kategorie 2'].str.contains(pat= 'Checkers Sixty60')), 'Sub-kategorie'] = 'Checkers'
# Fill NaN values met Weet Nie
df['Sub-kategorie'].fillna('Weet Nie', inplace= True)

# Create 'n kolom vir Kategorie
df.loc[(df['Sub-kategorie'].str.contains(pat= 'Pa')), 'Kategorie'] = 'FT'
df.loc[(df['Sub-kategorie'].str.contains(pat= 'BYC')), 'Kategorie'] = 'Savings'
df.loc[(df['Sub-kategorie'].str.contains(pat= 'Unknown Misc')), 'Kategorie'] = 'Misc'
df.loc[(df['Sub-kategorie'].str.contains(pat= 'Kruie')), 'Kategorie'] = 'Misc'
df.loc[(df['Sub-kategorie'].str.contains(pat= 'Pick-n-Pay')), 'Kategorie'] = 'Groceries'
df.loc[(df['Sub-kategorie'].str.contains(pat= 'Ma')), 'Kategorie'] = 'FT'
df.loc[(df['Sub-kategorie'].str.contains(pat= 'Kitskos')), 'Kategorie'] = 'Misc'
df.loc[(df['Sub-kategorie'].str.contains(pat= 'Alkoholiese Verversings')), 'Kategorie'] = 'Misc'
df.loc[(df['Sub-kategorie'].str.contains(pat= 'Clicks')), 'Kategorie'] = 'Groceries'
df.loc[(df['Sub-kategorie'].str.contains(pat= 'ATM')), 'Kategorie'] = 'Misc'
df.loc[(df['Sub-kategorie'].str.contains(pat= 'Anna')), 'Kategorie'] = 'Misc'
df.loc[(df['Sub-kategorie'].str.contains(pat= 'Elektrisiteit')), 'Kategorie'] = 'Utilities'
df.loc[(df['Sub-kategorie'].str.contains(pat= 'Water & Sanitation')), 'Kategorie'] = 'Utilities'
df.loc[(df['Sub-kategorie'].str.contains(pat= 'Depositos')), 'Kategorie'] = 'Misc'
df.loc[(df['Sub-kategorie'].str.contains(pat= 'Service Fee')), 'Kategorie'] = 'Bank Fees'
df.loc[(df['Sub-kategorie'].str.contains(pat= 'Apple')), 'Kategorie'] = 'Subscriptions'
df.loc[(df['Sub-kategorie'].str.contains(pat= 'Takealot')), 'Kategorie'] = 'Misc'
df.loc[(df['Sub-kategorie'].str.contains(pat= 'Checkers')), 'Kategorie'] = 'Groceries'
df.loc[(df['Sub-kategorie'].str.contains(pat= 'Couch')), 'Kategorie'] = 'Misc'
df.loc[(df['Sub-kategorie'].str.contains(pat= 'Woolworths')), 'Kategorie'] = 'Groceries'
df.loc[(df['Sub-kategorie'].str.contains(pat= 'Dischem')), 'Kategorie'] = 'Groceries'
df.loc[(df['Sub-kategorie'].str.contains(pat= 'Discovery')), 'Kategorie'] = 'Medies'
df.loc[(df['Sub-kategorie'].str.contains(pat= 'Yskas Repair')), 'Kategorie'] = 'Misc'
df.loc[(df['Sub-kategorie'].str.contains(pat= 'Huis Koop Expenses')), 'Kategorie'] = 'Misc'
df.loc[(df['Sub-kategorie'].str.contains(pat= 'Gautrain')), 'Kategorie'] = 'Travel'
df.loc[(df['Sub-kategorie'].str.contains(pat= 'DIY')), 'Kategorie'] = 'Misc'
df.loc[(df['Sub-kategorie'].str.contains(pat= 'Internet')), 'Kategorie'] = 'Utilities'
df.loc[(df['Sub-kategorie'].str.contains(pat= 'Kredietkaart')), 'Kategorie'] = 'Kredietkaart Betalings'
df.loc[(df['Sub-kategorie'].str.contains(pat= 'Kar Lening')), 'Kategorie'] = 'Kar'
df.loc[(df['Sub-kategorie'].str.contains(pat= 'Makro')), 'Kategorie'] = 'Misc'
df.loc[(df['Sub-kategorie'].str.contains(pat= 'Santam')), 'Kategorie'] = 'Versekering'
df.loc[(df['Sub-kategorie'].str.contains(pat= 'Cartrack')), 'Kategorie'] = 'Kar'
df.loc[(df['Sub-kategorie'].str.contains(pat= 'Petrol')), 'Kategorie'] = 'Travel'
df.loc[(df['Sub-kategorie'].str.contains(pat= 'Huur')), 'Kategorie'] = 'Huis'
df.loc[(df['Sub-kategorie'].str.contains(pat= 'FNB Account Fee')), 'Kategorie'] = 'Bank Fees'
df.loc[(df['Sub-kategorie'].str.contains(pat= 'FNB RA')), 'Kategorie'] = 'Retirement'
df.loc[(df['Sub-kategorie'].str.contains(pat= 'Erfbelasting')), 'Kategorie'] = 'Huis'
df.loc[(df['Sub-kategorie'].str.contains(pat= 'Rente')), 'Kategorie'] = 'Bank Fees'
df.loc[(df['Sub-kategorie'].str.contains(pat= 'Med-term Savings')), 'Kategorie'] = 'Savings'
df.loc[(df['Sub-kategorie'].str.contains(pat= 'FNB Credit Fee')), 'Kategorie'] = 'Bank Fees'
df.loc[(df['Sub-kategorie'].str.contains(pat= 'MB Printing')), 'Kategorie'] = 'Misc'
df.loc[(df['Sub-kategorie'].str.contains(pat= 'Huis Lening')), 'Kategorie'] = 'Huis'
df.loc[(df['Sub-kategorie'].str.contains(pat= 'IKH')), 'Kategorie'] = 'Misc'
df.loc[(df['Sub-kategorie'].str.contains(pat= 'Ilion')), 'Kategorie'] = 'Salaris'
df.loc[(df['Sub-kategorie'].str.contains(pat= 'Lynnwood Housewife')), 'Kategorie'] = 'Misc'
df.loc[(df['Sub-kategorie'].str.contains(pat= 'Food Lovers')), 'Kategorie'] = 'Groceries'
df.loc[(df['Sub-kategorie'].str.contains(pat= 'Vega Motors')), 'Kategorie'] = 'Misc'
df.loc[(df['Sub-kategorie'].str.contains(pat= 'Spar')), 'Kategorie'] = 'Groceries'
df.loc[(df['Sub-kategorie'].str.contains(pat= 'Spotify')), 'Kategorie'] = 'Subscriptions'
df.loc[(df['Sub-kategorie'].str.contains(pat= 'Weet Nie')), 'Kategorie'] = 'Misc'
df.loc[(df['Sub-kategorie'].str.contains(pat= 'Huurinkomste')), 'Kategorie'] = 'Huis'
df.loc[(df['Sub-kategorie'].str.contains(pat= 'Time Square Parking')), 'Kategorie'] = 'Misc'
df.loc[(df['Sub-kategorie'].str.contains(pat= 'Bertie')), 'Kategorie'] = 'FT'
df.loc[(df['Sub-kategorie'].str.contains(pat= 'Incredible Connection')), 'Kategorie'] = 'Misc'
df.loc[(df['Sub-kategorie'].str.contains(pat= 'Continental Linen')), 'Kategorie'] = 'Misc'
df.loc[(df['Sub-kategorie'].str.contains(pat= 'Petzone')), 'Kategorie'] = 'Pets'
df.loc[(df['Sub-kategorie'].str.contains(pat= 'MTN')), 'Kategorie'] = 'Foon'
df.loc[(df['Sub-kategorie'].str.contains(pat= 'Playstation Network')), 'Kategorie'] = 'Subscriptions'
df.loc[(df['Sub-kategorie'].str.contains(pat= 'Guardrisk')), 'Kategorie'] = 'Versekering'
df.loc[(df['Sub-kategorie'].str.contains(pat= 'Bolt')), 'Kategorie'] = 'Travel'
df.loc[(df['Sub-kategorie'].str.contains(pat= 'Debit Order Dispute Fee')), 'Kategorie'] = 'Bank Fees'
df.loc[(df['Sub-kategorie'].str.contains(pat= 'Declined Foreign Transaction')), 'Kategorie'] = 'Bank Fees'
df.loc[(df['Sub-kategorie'].str.contains(pat= 'Reversal')), 'Kategorie'] = 'Misc'
df.loc[(df['Sub-kategorie'].str.contains(pat= 'Midas')), 'Kategorie'] = 'Misc'


In [ ]:
# Partition dfs in maande in
# Create variables vir 'Year Month' kolom
days_of_month = df['Date'].dt.day
greater_than_24 = df['Date'] + pd.Timedelta(days=10)
less_than_24 = df['Date']
# Add kolom vir maand en jaar
df['Year Month'] = np.where(days_of_month > 24, greater_than_24.dt.strftime('%Y %b'), less_than_24.dt.strftime('%Y %b'))

# Loop deur df en save files volgense Year Month kolom
for filename in df['Year Month']:
  df_partitioned = df[df['Year Month'] == filename]
  df_partitioned.to_excel('/content/drive/MyDrive/Data Lake/Finances/Transaction History/FNB Cheque Account/Cleaned Data/' + filename + '.xlsx')
